In [1]:
pip install -q google-generativeai google-adk

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Configure a API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [15]:
#pip install -q google-generativeai google-adk
#Note: you may need to restart the kernel to use updated packages.

# Import necessary libraries
import os
from datetime import date
import textwrap
import warnings
import requests # This import was in the original code but not used in the provided functions. Kept for completeness.

# For Google Gemini and ADK
import google.generativeai as genai # Updated import for Gemini SDK
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- API Key Configuration ---
# In a real-world application, avoid hardcoding API keys.
# For VS Code, you can load it from an environment variable or a configuration file.
# For this example, we'll assume it's set as an environment variable.
# You can set it in your terminal before running the script:
# export GOOGLE_API_KEY='YOUR_API_KEY_HERE'
# Or, if using a .env file and a library like python-dotenv:
# from dotenv import load_dotenv
# load_dotenv()
# GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
#genai.configure(api_key=os.environ["GEMINI_API_KEY"])

""" if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY environment variable not set. Please set it before running the script.") """

# genai.configure(api_key=GOOGLE_API_KEY) # Configure the genai library with the API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Define the Gemini model ID
MODEL_ID = "gemini-1.5-flash" # Changed to 1.5-flash for broader availability and cost-effectiveness.
                              # Original was "gemini-2.0-flash", which might not be publicly available yet
                              # or require specific preview access.

# --- Auxiliary Functions ---

def call_agent(agent: Agent, message_text: str) -> str:
    """
    Auxiliary function to send a message to an agent via Runner and return the final response.
    """
    # Create an in-memory session service
    session_service = InMemorySessionService()
    # Create a new session (you can customize IDs as needed)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Create a Runner for the agent
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Create the input message content
    
    #content = genai.types.Content(role="user", parts=[genai.types.Part(text=message_text)])
    # AttributeError: module 'google.generativeai.types' has no attribute 'Content'

    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    # NameError: name 'types' is not defined; correção:
    # from google.genai import types  # Para criar conteúdos (Content e Part)


    final_response = ""
    # Iterate asynchronously through the events returned during agent execution
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

def to_markdown(text):
    """
    Auxiliary function to format text as Markdown for better readability in console or VS Code output.
    Replaces bullet points and indents the text.
    """
    text = text.replace('•', '  *')
    # Using print directly as 'display(Markdown(...))' is for Jupyter notebooks
    return textwrap.indent(text, '> ', predicate=lambda _: True)

# --- Agent Definitions ---

def agente_buscador(topico, data_de_hoje):
    """
    Agent 1: News Finder.
    Searches for recent and relevant news about product launches.
    """
    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (Google Search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Execute the agent
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

def agente_planejador(topico, lancamentos_buscados):
    """
    Agent 2: Post Planner.
    Creates a plan for an Instagram post based on recent launches.
    """
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (Google Search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (Google Search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Execute the agent
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

def agente_redator(topico, plano_de_post):
    """
    Agent 3: Post Writer.
    Writes a draft Instagram post based on the provided plan.
    """
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Execute the agent
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

def agente_revisor(topico, rascunho_gerado):
    """
    Agent 4: Quality Reviewer.
    Reviews the draft Instagram post for clarity, conciseness, correctness, and tone.
    """
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Execute the agent
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

# --- Main Program Logic ---

if __name__ == "__main__":
    # Get today's date
    data_de_hoje = date.today().strftime("%d/%m/%Y")

    print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

    # --- Obtain the Topic from the User ---
    topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

    if not topico:
        print("Por favor, insira um tópico válido.")
    else:
        print(f"Maravilha! Vamos então criar o post sobre: {topico}")

        # Step 1: Agente Buscador (News Finder Agent)
        print("\n--- AGENTE BUSCADOR: Buscando lançamentos recentes ---")
        lancamentos = agente_buscador(topico, data_de_hoje)
        print(to_markdown(lancamentos))
        print("--------------------------------------------------------------")

        # Step 2: Agente Planejador (Post Planner Agent)
        print("\n--- AGENTE PLANEJADOR: Criando o plano do post ---")
        plano = agente_planejador(topico, lancamentos)
        print(to_markdown(plano))
        print("--------------------------------------------------------------")

        # Step 3: Agente Redator (Post Writer Agent)
        print("\n--- AGENTE REDATOR: Escrevendo o rascunho do post ---")
        rascunho = agente_redator(topico, plano)
        print(to_markdown(rascunho))
        print("--------------------------------------------------------------")

        # Step 4: Agente Revisor (Quality Reviewer Agent) - Corrected flow
        print("\n--- AGENTE REVISOR: Revisando o rascunho do post ---")
        texto_revisado = agente_revisor(topico, rascunho) # Corrected agent call
        print(to_markdown(texto_revisado))
        print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀


Exception in thread Thread-19 (_asyncio_thread_main):
Traceback (most recent call last):
  File "C:\Program Files\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\google\adk\runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\base_events.py", line 691, in run_until_complete
    r

Maravilha! Vamos então criar o post sobre: manutençao iof

--- AGENTE BUSCADOR: Buscando lançamentos recentes ---

--------------------------------------------------------------

--- AGENTE PLANEJADOR: Criando o plano do post ---


Exception in thread Thread-21 (_asyncio_thread_main):
Traceback (most recent call last):
  File "C:\Program Files\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\google\adk\runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\base_events.py", line 691, in run_until_complete
    r


--------------------------------------------------------------

--- AGENTE REDATOR: Escrevendo o rascunho do post ---

--------------------------------------------------------------

--- AGENTE REVISOR: Revisando o rascunho do post ---

--------------------------------------------------------------


In [20]:
# Import necessary libraries
import os
from datetime import date
import textwrap
import warnings
import requests

# For Google Gemini and ADK
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- API Key Configuration ---
# In a real-world application, avoid hardcoding API keys.
# For VS Code, you can load it from an environment variable.
# You can set it in your terminal before running the script:
# export GOOGLE_API_KEY='YOUR_API_KEY_HERE'
""" if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY environment variable not set. Please set it before running the script.") """

# genai.configure(api_key=GOOGLE_API_KEY) # Configure the genai library with the API key
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Define the Gemini model ID
MODEL_ID = "gemini-1.5-flash" # Using a generally available model

# --- Auxiliary Functions ---

def call_agent(agent: Agent, message_text: str, session_service: InMemorySessionService, user_session_id: str) -> str:
    """
    Auxiliary function to send a message to an agent via Runner and return the final response.
    It now accepts an existing session_service and a user_session_id to maintain session state.
    """
    try:
        # Try to create a new session with the given ID.
        # If a session with this ID already exists, it will raise a ValueError.
        session = session_service.create_session(app_name=agent.name, user_id="user1", session_id=user_session_id)
    except ValueError:
        # If the session already exists (e.g., from a previous agent call using the same ID),
        # retrieve the existing session instead of trying to create it again.
        session = session_service.get_session(app_name=agent.name, user_id="user1", session_id=user_session_id)

    # Create a Runner for the agent, using the persistent session_service
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Create the input message content
    #content = genai.types.Content(role="user", parts=[genai.types.Part(text=message_text)])
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Iterate asynchronously through the events returned during agent execution
    # Ensure the same user_session_id is used for the runner.run call
    for event in runner.run(user_id="user1", session_id=user_session_id, new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

def to_markdown(text):
    """
    Auxiliary function to format text as Markdown for better readability in console or VS Code output.
    Replaces bullet points and indents the text.
    """
    text = text.replace('•', '  *')
    return textwrap.indent(text, '> ', predicate=lambda _: True)

# --- Agent Definitions ---
# Agent functions now accept session_service and user_session_id
# These parameters are passed down to the call_agent function

def agente_buscador(topico, data_de_hoje, session_service, user_session_id):
    """
    Agent 1: News Finder.
    Searches for recent and relevant news about product launches using Google Search.
    """
    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (Google Search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador, session_service, user_session_id)
    return lancamentos_buscados

def agente_planejador(topico, lancamentos_buscados, session_service, user_session_id):
    """
    Agent 2: Post Planner.
    Creates a plan for an Instagram post based on recent launches, using Google Search for more info.
    """
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (Google Search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (Google Search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador, session_service, user_session_id)
    return plano_do_post

def agente_redator(topico, plano_de_post, session_service, user_session_id):
    """
    Agent 3: Post Writer.
    Writes a draft Instagram post based on the provided plan.
    """
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    rascunho = call_agent(redator, entrada_do_agente_redator, session_service, user_session_id)
    return rascunho

def agente_revisor(topico, rascunho_gerado, session_service, user_session_id):
    """
    Agent 4: Quality Reviewer.
    Reviews the draft Instagram post for clarity, conciseness, correctness, and tone.
    """
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor, session_service, user_session_id)
    return texto_revisado

# --- Main Program Logic ---

if __name__ == "__main__":
    # Get today's date
    data_de_hoje = date.today().strftime("%d/%m/%Y")

    print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

    # --- Initialize InMemorySessionService once ---
    # This is crucial for maintaining session state across multiple agent calls.
    shared_session_service = InMemorySessionService()
    # Define a consistent session ID for this entire interaction flow.
    main_session_id = "user_main_session_flow"

    # --- Obtain the Topic from the User ---
    topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

    if not topico:
        print("Por favor, insira um tópico válido.")
    else:
        print(f"Maravilha! Vamos então criar o post sobre: {topico}")

        # Step 1: Agente Buscador (News Finder Agent)
        print("\n--- AGENTE BUSCADOR: Buscando lançamentos recentes ---")
        # Pass the shared_session_service and main_session_id to each agent function
        lancamentos = agente_buscador(topico, data_de_hoje, shared_session_service, main_session_id)
        print(to_markdown(lancamentos))
        print("--------------------------------------------------------------")

        # Step 2: Agente Planejador (Post Planner Agent)
        print("\n--- AGENTE PLANEJADOR: Criando o plano do post ---")
        plano = agente_planejador(topico, lancamentos, shared_session_service, main_session_id)
        print(to_markdown(plano))
        print("--------------------------------------------------------------")

        # Step 3: Agente Redator (Post Writer Agent)
        print("\n--- AGENTE REDATOR: Escrevendo o rascunho do post ---")
        rascunho = agente_redator(topico, plano, shared_session_service, main_session_id)
        print(to_markdown(rascunho))
        print("--------------------------------------------------------------")

        # Step 4: Agente Revisor (Quality Reviewer Agent)
        print("\n--- AGENTE REVISOR: Revisando o rascunho do post ---")
        # Ensure the correct agent (revisor) is called with the rascunho
        texto_revisado = agente_revisor(topico, rascunho, shared_session_service, main_session_id)
        print(to_markdown(texto_revisado))
        print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀


Exception in thread Thread-23 (_asyncio_thread_main):
Traceback (most recent call last):
  File "C:\Program Files\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\google\adk\runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\base_events.py", line 691, in run_until_complete
    r

Maravilha! Vamos então criar o post sobre: iof mantida

--- AGENTE BUSCADOR: Buscando lançamentos recentes ---

--------------------------------------------------------------

--- AGENTE PLANEJADOR: Criando o plano do post ---


Exception in thread Thread-24 (_asyncio_thread_main):
Traceback (most recent call last):
  File "C:\Program Files\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Issao\GitHub\Venv_01\.venv\Lib\site-packages\google\adk\runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\asyncio\base_events.py", line 691, in run_until_complete
    r


--------------------------------------------------------------

--- AGENTE REDATOR: Escrevendo o rascunho do post ---

--------------------------------------------------------------

--- AGENTE REVISOR: Revisando o rascunho do post ---

--------------------------------------------------------------
